# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# Notebook 10 — E2E: User Long-Term Memory (LTM)
This notebook validates the engine-integrated user long-term memory path end-to-end:
- persistence (profile → JSON snapshot),
- modifications (edit + soft delete),
- semantic retrieval via `SessionManager.search_user_longterm_memory(...)`,
- engine injection via `_step_user_longterm_memory` (SYSTEM message),
- multi-session simulation (Session 1 → Session 2) without refactors.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from intergrax.llm_adapters.base import LLMAdapter, LLMAdapterRegistry, LLMProvider
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager
from intergrax.memory.user_profile_manager import UserProfileManager
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager

# ---------------------------------------------------------------------
# Test identifiers
# ---------------------------------------------------------------------
USER_ID = "user_e2e_ltm_001"
SESSION_1 = "sess_e2e_ltm_001"
SESSION_2 = "sess_e2e_ltm_002"

# ---------------------------------------------------------------------
# LLM adapter (real adapter, no wrappers)
# - assumes env is configured (OPENAI_API_KEY etc.)
# ---------------------------------------------------------------------
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)


# ---------------------------------------------------------------------
# Embeddings + vectorstore (real managers)
# Pick providers you actually use in your repo/env.
# ---------------------------------------------------------------------

embed_manager = EmbeddingManager(
    verbose=True,
    provider="ollama",
)

vectorstore_manager = VectorstoreManager(
    config=VSConfig(
        provider="chroma",
        collection_name="intergrax_user_ltm",
    ),
    verbose=True,
)

# ---------------------------------------------------------------------
# Stores
# ---------------------------------------------------------------------
session_store = InMemorySessionStorage()
user_profile_store = InMemoryUserProfileStore()

# ---------------------------------------------------------------------
# Managers
# ---------------------------------------------------------------------
user_profile_manager = UserProfileManager(
    store=user_profile_store,
    embedding_manager=embed_manager,
    vectorstore_manager=vectorstore_manager,
)

session_manager = SessionManager(
    storage=session_store,
    user_profile_manager=user_profile_manager,
)

# ---------------------------------------------------------------------
# Runtime config (LTM enabled, RAG disabled to isolate the feature)
# ---------------------------------------------------------------------
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_user_profile_memory=True,
    enable_org_profile_memory=False,
    enable_user_longterm_memory=True,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

runtime = DropInKnowledgeRuntime(
    config=config,
    session_manager=session_manager,
    ingestion_service=None,
    context_builder=None,
    rag_prompt_builder=None,
    websearch_prompt_builder=None,
    history_prompt_builder=None,
)

print("BOOTSTRAP OK")
print("USER_ID:", USER_ID)
print("SESSION_1:", SESSION_1)
print("SESSION_2:", SESSION_2)



[intergraxVectorstoreManager] Initialized provider=chroma, collection=intergrax_user_ltm
[intergraxVectorstoreManager] Existing count: 3
BOOTSTRAP OK
USER_ID: user_e2e_ltm_001
SESSION_1: sess_e2e_ltm_001
SESSION_2: sess_e2e_ltm_002


# (Code cell) STEP 1: Seed LTM entries + search (UserProfileManager only)

In [3]:
from intergrax.memory.user_profile_memory import UserProfileMemoryEntry, MemoryKind, MemoryImportance

async def step_1_seed_and_search() -> None:
    # 1) Ensure session exists (SessionManager is the gate)        
    await session_manager.get_or_create_session(user_id=USER_ID, session_id=SESSION_1)

    # 2) Create deterministic LTM entries
    e1 = UserProfileMemoryEntry(
        content="User builds Intergrax and Mooff. Goal: ChatGPT-like runtime with memory, RAG, tools, multi-session.",
        session_id=SESSION_1,
        kind=MemoryKind.USER_FACT,
        importance=MemoryImportance.HIGH,
        title="Identity: Intergrax/Mooff",
        metadata={"tags": ["intergrax", "mooff", "runtime"]},
    )

    e2 = UserProfileMemoryEntry(
        content="User prefers concise, technical answers. Never use emojis in code/docs.",
        session_id=SESSION_1,
        kind=MemoryKind.PREFERENCE,
        importance=MemoryImportance.HIGH,
        title="Preference: Style",
        metadata={"tags": ["style", "formatting"]},
    )

    e3 = UserProfileMemoryEntry(
        content="Architecture decision: UserProfileManager owns LTM index; engine uses _step_user_longterm_memory.",
        session_id=SESSION_1,
        kind=MemoryKind.OTHER,
        importance=MemoryImportance.MEDIUM,
        title="Decision: LTM ownership",
        metadata={"tags": ["ltm", "architecture"]},
    )

    # 3) Persist + index
    await user_profile_manager.add_memory_entry(USER_ID, e1)
    await user_profile_manager.add_memory_entry(USER_ID, e2)
    await user_profile_manager.add_memory_entry(USER_ID, e3)

    # 4) Search (explicitly no threshold)
    res = await user_profile_manager.search_longterm_memory(
        user_id=USER_ID,
        query="What am I building and what is my preferred answer style?",
        top_k=5,
        score_threshold=None,
    )

    dbg = res.get("debug") or {}
    print("STEP 1 RESULT")
    print("- debug.enabled :", dbg.get("enabled"))
    print("- debug.used    :", dbg.get("used"))
    print("- hits_count    :", dbg.get("hits_count"))
    print("- reason        :", dbg.get("reason"))

    hits = res.get("hits") or []
    scores = res.get("scores") or []
    for h, s in zip(hits, scores):
        print(f"  - entry_id={h.entry_id} score={s:.4f} title={h.title!r} deleted={getattr(h, 'deleted', False)}")

await step_1_seed_and_search()


[intergraxVectorstoreManager] Upserting 1 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 4
[intergraxVectorstoreManager] Upserting 1 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 5
[intergraxVectorstoreManager] Upserting 1 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 6
STEP 1 RESULT
- debug.enabled : True
- debug.used    : True
- hits_count    : 3
- reason        : hits
  - entry_id=70964af4d7244cb5a4413fdc10243af4 score=-0.1371 title='Preference: Style' deleted=False
  - entry_id=6396b7df62e94c55b4a847c55ab1d01d score=-0.2065 title='Identity: Intergrax/Mooff' deleted=False
  - entry_id=14164063a9614cf2ac9b616c35b75531 score=-0.4516 title='Decision: LTM ownership' deleted=False


# (Code cell) STEP 2: Update + soft delete + re-search

In [4]:
async def step_2_update_delete_research() -> None:
    # 1) Get current hits so we have stable entry_ids
    res1 = await user_profile_manager.search_longterm_memory(
        user_id=USER_ID,
        query="What is my preferred answer style?",
        top_k=5,
        score_threshold=None,
    )

    hits1 = res1.get("hits") or []
    if len(hits1) < 2:
        raise RuntimeError(f"Expected at least 2 hits, got {len(hits1)}")

    # Choose:
    # - update the top hit (style preference)
    # - delete the second hit (identity or similar)
    entry_to_update = hits1[0]
    entry_to_delete = hits1[1]

    # 2) Update (append deterministic marker)
    updated_text = (entry_to_update.content or "") + " [UPDATED_IN_NOTEBOOK_STEP_2]"
    entry_to_update.content = updated_text
    entry_to_update.modified = True

    # Persist update (expects your manager/store to handle modified flag)
    await user_profile_manager.update_memory_entry(USER_ID, entry_to_update)

    # 3) Soft delete
    entry_to_delete.deleted = True
    await user_profile_manager.remove_memory_entry(USER_ID, entry_to_delete.entry_id)

    # 4) Re-search: updated marker should be retrievable, deleted entry must not appear
    res2 = await user_profile_manager.search_longterm_memory(
        user_id=USER_ID,
        query="UPDATED_IN_NOTEBOOK_STEP_2",
        top_k=10,
        score_threshold=None,
    )

    dbg2 = res2.get("debug") or {}
    print("STEP 2 RESULT")
    print("- debug.used  :", dbg2.get("used"))
    print("- hits_count  :", dbg2.get("hits_count"))
    print("- reason      :", dbg2.get("reason"))

    hits2 = res2.get("hits") or []
    ids2 = [h.entry_id for h in hits2]

    print("- updated_entry_id:", entry_to_update.entry_id, "FOUND" if entry_to_update.entry_id in ids2 else "NOT_FOUND")
    print("- deleted_entry_id:", entry_to_delete.entry_id, "FOUND" if entry_to_delete.entry_id in ids2 else "NOT_FOUND (OK)")

    for h, s in zip(res2.get("hits") or [], res2.get("scores") or []):
        print(f"  - entry_id={h.entry_id} score={s:.4f} title={h.title!r} deleted={getattr(h, 'deleted', False)}")

await step_2_update_delete_research()


STEP 2 RESULT
- debug.used  : True
- hits_count  : 2
- reason      : hits
- updated_entry_id: 70964af4d7244cb5a4413fdc10243af4 FOUND
- deleted_entry_id: 6396b7df62e94c55b4a847c55ab1d01d NOT_FOUND (OK)
  - entry_id=14164063a9614cf2ac9b616c35b75531 score=-0.1545 title='Decision: LTM ownership' deleted=False
  - entry_id=70964af4d7244cb5a4413fdc10243af4 score=-0.4100 title='Preference: Style' deleted=False


# (Code cell) STEP 3: Session 2 -> runtime.run() and verify debug trace contains LTM usage

In [11]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest


async def step_3_runtime_session_2() -> None:
    # 1) Ensure session 2 exists
    await session_manager.get_or_create_session(user_id=USER_ID, session_id=SESSION_2)

    # 2) Build canonical request
    req = RuntimeRequest(
        user_id=USER_ID,
        session_id=SESSION_2,
        message="Remind me what I'm building and what answer style I prefer. Include the UPDATED_IN_NOTEBOOK_STEP_2 marker if present.",
        attachments=[],   # no attachments in this test
    )

    # 3) Run runtime
    ans = await runtime.run(req)

    # 4) Print answer (use the canonical field names from your RuntimeAnswer)
    print("STEP 3 ANSWER")
    print(ans.answer)

    print("STEP 3.1 TYPE CHECK")
    print("type(ans.answer) =", type(ans.answer))

    print("STEP 3.1 ANSWER TEXT")
    print(ans.answer)
    print("answer_len =", len(ans.answer))
    print("answer_preview =", repr(ans.answer[:200]))

    # Also print a shortened repr to see what's inside
    print("repr(ans.answer) =", repr(ans.answer))

    # If your session manager exposes session/history, read it to see what was persisted.
    # Adjust names to your actual API if needed.
    sess2 = await session_manager.get_session(session_id=SESSION_2)
    history = await session_manager.get_history(session_id=sess2.id)
    print("\nSTEP 3.1 SESSION 2 HISTORY (last 6)")
    for m in history[-6:]:
        print(f"- {m.role}: {m.content[:200]!r}")

    # 5) Verify debug trace (canonical fields, no hasattr)
    dbg = ans.debug_trace
    ltm = dbg.get("user_longterm_memory")

    print("\nSTEP 3 DEBUG")
    print("- ltm.used      :", ltm.get("used"))
    print("- ltm.reason    :", ltm.get("reason"))
    print("- ltm.hits_count:", ltm.get("hits_count"))

    # Optional: assert-like checks (fail fast)
    if not ltm.get("used"):
        raise RuntimeError(f"LTM not used. Reason: {ltm.get('reason')}")
    if (ltm.get("hits_count") or 0) <= 0:
        raise RuntimeError("LTM used=True but hits_count<=0 (unexpected).")


await step_3_runtime_session_2()


STEP 3 ANSWER
content='' additional_kwargs={} response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-12-18T15:51:19.1444991Z', 'done': True, 'done_reason': 'stop', 'total_duration': 508742200, 'load_duration': 404404000, 'prompt_eval_count': 1072, 'prompt_eval_duration': 84641100, 'eval_count': 1, 'eval_duration': None, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None)} id='run--83f35ee3-f138-42ae-84e5-c4c4d579bdaf-0' usage_metadata={'input_tokens': 1072, 'output_tokens': 1, 'total_tokens': 1073}
STEP 3.1 TYPE CHECK
type(ans.answer) = <class 'str'>
STEP 3.1 ANSWER TEXT
content='' additional_kwargs={} response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-12-18T15:51:19.1444991Z', 'done': True, 'done_reason': 'stop', 'total_duration': 508742200, 'load_duration': 404404000, 'prompt_eval_count': 1072, 'prompt_eval_duration': 84641100, 'eval_count': 1, 'eval_duration': None, 'message': Message(role='assista